In [1]:
# this imports the code written in the src directory
import sys
sys.path.insert(0, '../')

import numpy as np

# TSP Trajectory generation using node placements

This is Algorithm 4.7.2 from Rubinstein and Kroese, *The Cross-Entropy Method: A Unified Approach to Combinatorial Optimization, Monte-Carlo Simulation, and Machine Learning* (2004). Given $n$ points in the plane $\mathbb{R}^2$, we want to construct a TSP tour. Consider the point labels $[0,1,\dots,n-1]$. It suffices to construct the tour on these labels. The idea from the book is as follows: suppose we have a discrete-time Markov chain on the labels with a transition probability matrix $P$. Starting from state $0$, draw the next state according to the distribution $P_{0, :}$. But in order to make sure that we don't accidentally choose $0$ again (invalid as a TSP tour), we negate the column $0$ and renormalize the entire matrix $P$. Then we proceed by drawing the next state. In general, given that we are at state $j$, we negate the $j$ column and renormalize $P$. Then we draw the next state from $P_{j,:}$. After we have done this $n-1$ times, we are finished, because we then must return to state $0$. In pseudocode, it resembles:

1. Define $P^{(0)} = P$. Let $j = 0$.
2. Generate $X_j$ from the distribution formed by the $j$th row of $P^{(j)}$. Obtain the matrix $P^{(j+1)}$ from $P^{(j)}$ by first setting the $X_j$th column of $P^{(j)}$ to $0$ and then normalizing the rows to sum up to $1$.
3. If $j = n-1$, stop. Otherwise, set  $j = j + 1$ and repeat 2.

## Relevance

For the Cross-Entropy method, we need to create a way to parameterize the class of probability distributions over the space of tours. While there is a natural way to do so when we assume at the outset that all tours are equally useful---and hence probable---for CE to work we need to be able to adjust the distribution so that it emphasizes the better-scoring tours. Introducing the transition probability matrix $P$ gives us exactly this feature. Now we can adjust the likelihood of drawing a given tour depending on how successful that particular tour shows itself to be during the simulation.

Here is an implementation of the TSP trajectory generation technique:

In [2]:
def normalize_nonnegative_matrix(matrix, axis):
    """
    Given a nonnegative matrix P and an axis (either 0 -- for normalizing along columns -- or 1 -- for normalizing along rows), normalize the matrix.
    
    This is an inplace transformation: it modifies the original input matrix.
    """
    matrix /= np.sum(matrix, axis=axis)[:,None]

    
def draw_from(distribution, size=1):
    """
    Given a finite distribution [p0, p1, ... pn-1] (here pj is the probability of drawing j) such that sum(pj for pj in distribution) == 1,
    draw a random variable X in [0, 1, ... n-1] which has the pmf of the distribution.
    
    The way it works is as follows. First, the cumulative distribution function is computed. Then, a uniform random variate U ~ U(0,1) is drawn. 
    We find the largest j such that U < cdf(j), and return it.
    """
    
    # The actual code involves some abuse of NumPy
    # return np.argmax(1 - (np.cumsum(distribution) < np.random.rand()))
    return np.random.choice(np.arange(len(distribution)), size=size, p=distribution)


def generate_trajectory(transition_matrix):
    """
    Generate a trajectory on the points [0, 1, ..., n-1] in accordance with the transition matrix of a Markov chain on these points. This method follows
    the algorithm in (Rubinstein and Kroese 2004, Algorithm 4.7.2)
    """
    n = transition_matrix.shape[0]
    matrix = transition_matrix.copy()
    trajectory = [0]
    for j in range(len(matrix)-1):
        matrix[:,trajectory[j]] = 0.
        normalize_nonnegative_matrix(matrix, 1)
        trajectory.append(np.asscalar(draw_from(matrix[trajectory[j],:].flatten())))

    return trajectory

In [3]:
# Demonstration
n = 12
P = np.random.rand(n,n) # P[i,j] ~ Unif(0,1)
normalize_nonnegative_matrix(P, axis=1)

In [4]:
generate_trajectory(P)

[0, 8, 2, 4, 3, 9, 7, 11, 1, 10, 6, 5]

# $k$-Drone Trajectory generation using node placements

The challenge for us is to generalize this model for $k$-drone tours. Again, for CE to work well, we need to be able to adjust the distribution over the space of $k$-drone tours so that as the simulation proceeds, better $k$-drone tours are given higher preference. We would like a similar parameterization to the TSP ($1$-drone tour) case above, but that incorporates the added flexibility of multiple tours.

Here's what I propose to be the new algorithm. Given input of a transition probability matrix $P$ *and* a probabilty distribution on $[0, 1, \dots, n-1]$ denoted $[p_0, \dots, p_{k-1}]$, we proceed with the trajectory genreation but with a modification. First, we select $k$ starting positions according to the probabilitiy distribution on the labels. Next, before we determine the next step in tours, we roll a $k$-sided die. Whichever drone $i$ the die lands on, we perform Step (2) from the original algorithm. Then we proceed as before.

1. Define $P^{(0)} = P$. Let $j = 0$. Set $X_{0,0}, \dots, X_{k-1,0}$ as a simple random sample according to the distribution $[p_0, \dots, p_{k-1}]$.
2. Roll a $k$-sided die. Let $i$ be the result.
3. Generate $X_{i,-1}$ from the distribution formed by the $i$th row of $P^{(j)}$. Obtain the matrix $P^{(j+1)}$ from $P^{(j)}$ by first setting the $X_{i,-1}$th column of $P^{(j)}$ to $0$ and then normalizing the rows to sum up to $1$.
4. If $j = n-1$, stop. Otherwise, set  $j = j + 1$ and repeat 2.

In [5]:
def generate_k_trajectory(transition_matrix, start_dist, k):
    """
    Generates a k trajectory according to a specified transition probability matrix for a discrete-time Markov chain, a probability distribution over the starting sites, and a specified number of drones. 
    It works more or less as before in the 1-drone example, but now we have control over how many drones are in play as well as their initial locations.
    """
    n = transition_matrix.shape[0]
    matrix = transition_matrix.copy()
    
    starts = draw_from(start_dist, size=k)
    trajectories = {"Drone {0:02d}".format(i): [start] for (i, start) in enumerate(starts)}
    for (i, start) in enumerate(starts):
        matrix[:, start] = 0.
    normalize_nonnegative_matrix(matrix, 1)
    
    for j in range(len(matrix)-k):
        i = "Drone {0:02d}".format(np.random.randint(0, k))
        trajectories[i].append(np.asscalar(draw_from(matrix[trajectories[i][-1], :].flatten())))
        if j < len(matrix) - k-1:
            matrix[:, trajectories[i][-1]] = 0.
            normalize_nonnegative_matrix(matrix, 1)

    return trajectories

In [6]:
# Demonstration
n = 200
P = np.random.rand(n, n)
starts = np.random.rand(n)
starts /= np.sum(starts)
normalize_nonnegative_matrix(P, 1)

In [7]:
generate_k_trajectory(P, starts, 100)

{'Drone 00': [87],
 'Drone 01': [152],
 'Drone 02': [2],
 'Drone 03': [103, 109],
 'Drone 04': [151, 61],
 'Drone 05': [62],
 'Drone 06': [106],
 'Drone 07': [116, 67, 10, 30, 143],
 'Drone 08': [126, 46, 9],
 'Drone 09': [45, 74, 149],
 'Drone 10': [142, 51, 195],
 'Drone 11': [189],
 'Drone 12': [90, 18, 34],
 'Drone 13': [83, 167, 199],
 'Drone 14': [130],
 'Drone 15': [177, 47, 159],
 'Drone 16': [50, 124],
 'Drone 17': [84, 162, 6],
 'Drone 18': [13, 92],
 'Drone 19': [185],
 'Drone 20': [112, 66, 136],
 'Drone 21': [32, 125],
 'Drone 22': [53, 5],
 'Drone 23': [158, 42, 170],
 'Drone 24': [133],
 'Drone 25': [90, 85, 169],
 'Drone 26': [37],
 'Drone 27': [35],
 'Drone 28': [144],
 'Drone 29': [20, 79, 123],
 'Drone 30': [75],
 'Drone 31': [29, 89],
 'Drone 32': [58, 44, 128, 137, 175, 26],
 'Drone 33': [2, 4, 113],
 'Drone 34': [171, 180],
 'Drone 35': [45, 179],
 'Drone 36': [22, 99],
 'Drone 37': [105],
 'Drone 38': [97, 31],
 'Drone 39': [25],
 'Drone 40': [151, 176],
 'Drone 

# Next steps

We need to come up with a parameter updating formula for the parameters $P, q$ which fully describe the space of all $k$-tours.